In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import KFold, cross_val_score
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.model_selection import KFold
from sklearn.metrics import make_scorer, accuracy_score,roc_auc_score,roc_curve, auc,classification_report
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense
from keras.regularizers import l1
from tensorflow import keras
from sklearn.ensemble import RandomForestClassifier
import tensorflow as tf

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
data = pd.read_csv('/content/drive/MyDrive/DMPA_Project/Indian Liver Patient Dataset (ILPD).csv')

In [ ]:
col_names = ['Age','Gender','TB_total_bilirubin', 'DB_Direct_Bilirubin','Alkphos_Alkaline_Phosphotase', 'Sgpt_Alamine_Aminotransferase',
       'Sgot_Aspartate_Aminotransferase', 'TP_Total_Protiens', 'ALB_Albumin', 'A/G_Ratio','Selector Field']
len(col_names)

11

In [ ]:
data.columns = col_names

In [ ]:
data.isnull().sum()

Age                                0
Gender                             0
TB_total_bilirubin                 0
DB_Direct_Bilirubin                0
Alkphos_Alkaline_Phosphotase       0
Sgpt_Alamine_Aminotransferase      0
Sgot_Aspartate_Aminotransferase    0
TP_Total_Protiens                  0
ALB_Albumin                        0
A/G_Ratio                          4
Selector Field                     0
dtype: int64

In [ ]:
data['A/G_Ratio'].fillna(round(data['A/G_Ratio'].mean(),2),inplace=True)

In [ ]:
data.isnull().sum()

Age                                0
Gender                             0
TB_total_bilirubin                 0
DB_Direct_Bilirubin                0
Alkphos_Alkaline_Phosphotase       0
Sgpt_Alamine_Aminotransferase      0
Sgot_Aspartate_Aminotransferase    0
TP_Total_Protiens                  0
ALB_Albumin                        0
A/G_Ratio                          0
Selector Field                     0
dtype: int64

In [ ]:
data['Selector Field'] = data['Selector Field'].map(lambda x:0 if x==2 else 1)  # 0-no disease, 1-disease
data['Gender'] = data['Gender'].map(lambda x:0 if x=="Male" else 1)
value_table = pd.crosstab(data['Gender'],data['Selector Field'])
value_table.columns = ["No Disease","Disease Present"]
value_table.index = ["Male","Female"]
print(value_table)

        No Disease  Disease Present
Male           117              324
Female          50               91


In [ ]:
X = data.drop('Selector Field',axis=1)
y = data['Selector Field']

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_Scaled = scaler.fit_transform(X)

In [ ]:
Xtrain, Xtest, y_train, y_test = train_test_split(X,y,test_size=0.3,random_state=1)

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train = scaler.fit_transform(Xtrain)
X_test = scaler.transform(Xtest)

print(len(X_train))
print(len(y_train))

407
407


In [ ]:
pip install deap

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.4/135.4 kB 2.7 MB/s eta 0:00:00


GENETIC ALGORITHM

In [ ]:
import random
import numpy as np
from deap import base, creator, tools, algorithms
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

# Define the number of features (genes) to select
num_features_to_select = X.shape[1]  # You can adjust this number

# Create a fitness function for classification accuracy
creator.create("FitnessMax", base.Fitness, weights=(1.0,))
creator.create("Individual", list, fitness=creator.FitnessMax)

# Initialize the toolbox
toolbox = base.Toolbox()
toolbox.register("attr_bool", random.choice, [True, False])
toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.attr_bool, n=num_features_to_select)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

# Define the evaluation function (fitness function)
def evaluate(individual):
    selected_features = [feature for feature, select in zip(X_train.T, individual) if select]
    if not any(individual) or len(selected_features) == 0:
        return 0.0,  # If no features are selected, return a fitness of 0
    clf = RandomForestClassifier(random_state=0)
    clf.fit(X_train[:, individual], y_train)
    y_pred = clf.predict(X_test[:, individual])
    accuracy = accuracy_score(y_test, y_pred)
    return accuracy,

toolbox.register("evaluate", evaluate)
toolbox.register("mate", tools.cxTwoPoint)
toolbox.register("mutate", tools.mutFlipBit, indpb=0.05)
toolbox.register("select", tools.selTournament, tournsize=3)

# Create the initial population
population = toolbox.population(n=50)  # You can adjust the population size

# Run the genetic algorithm
generations = 10  # You can adjust the number of generations
for gen in range(generations):
    offspring = algorithms.varAnd(population, toolbox, cxpb=0.7, mutpb=0.3)
    fits = toolbox.map(toolbox.evaluate, offspring)
    for fit, ind in zip(fits, offspring):
        ind.fitness.values = fit
    population = toolbox.select(offspring, k=len(population))

# Find and print the best individual (selected features)
best_individual = tools.selBest(population, k=1)[0]
selected_features_indices = [i for i, select in enumerate(best_individual) if select]
print("Best Individual (Feature Selection):", selected_features_indices)

Best Individual (Feature Selection): [0, 2, 3, 5]


ARTIFICIAL NEURAL NETWORK

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import regularizers


classifier = Sequential()
classifier.add(Dense(units=256, activation='relu', input_dim=len(selected_features_indices)))
classifier.add(Dropout(rate=0.5))
classifier.add(Dense(units=128, activation='relu'))
classifier.add(Dropout(rate=0.5))
classifier.add(Dense(units=64, activation='relu'))
classifier.add(Dense(units=32, activation='relu'))
classifier.add(Dense(units=1, activation='sigmoid'))


optimizer = Adam(learning_rate=0.00001)
classifier.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
classifier.summary()


Model: "sequential_10"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_50 (Dense)            (None, 256)               1280      
                                                                 
 dropout_30 (Dropout)        (None, 256)               0         
                                                                 
 dense_51 (Dense)            (None, 128)               32896     
                                                                 
 dropout_31 (Dropout)        (None, 128)               0         
                                                                 
 dense_52 (Dense)            (None, 64)                8256      
                                                                 
 dense_53 (Dense)            (None, 32)                2080      
                                                                 
 dense_54 (Dense)            (None, 1)               

In [ ]:
X_new = X_train[:, selected_features_indices]
selected_columns = data.columns[selected_features_indices]
X_new = pd.DataFrame(X_new, columns=selected_columns)
print(X_new.head())

        Age  TB_total_bilirubin  DB_Direct_Bilirubin  \
0  0.197830            0.012221             0.057341   
1  0.686922            2.744035             2.686936   
2  0.625785            1.462908             1.335616   
3  1.481696           -0.421102            -0.453969   
4  1.848515            0.671624             0.787784   

   Sgpt_Alamine_Aminotransferase  
0                      -0.269058  
1                      -0.219737  
2                      -0.195077  
3                      -0.273990  
4                       0.544732  


In [ ]:
X_newtest = X_test[:, selected_features_indices]
selected_columns = data.columns[selected_features_indices]
X_newtest = pd.DataFrame(X_newtest, columns=selected_columns)
print(X_newtest.head())

        Age  TB_total_bilirubin  DB_Direct_Bilirubin  \
0 -1.147173            0.690464             0.641695   
1 -1.147173           -0.345741            -0.380925   
2 -0.352399           -0.232701            -0.198314   
3  0.931468           -0.213861            -0.125270   
4 -0.291262            2.141152             1.810404   

   Sgpt_Alamine_Aminotransferase  
0                      -0.249329  
1                      -0.244397  
2                      -0.224669  
3                      -0.175349  
4                      -0.185213  


In [ ]:
epochs = 50
batch_size = 50
history = classifier.fit(X_new, y_train, epochs=epochs, batch_size=batch_size, shuffle=True, validation_split=0.1)

Epoch 1/50
8/8 [==============================] - 0s 31ms/step - loss: 0.5929 - accuracy: 0.6995 - val_loss: 0.5849 - val_accuracy: 0.7805
Epoch 2/50
8/8 [==============================] - 0s 17ms/step - loss: 0.5962 - accuracy: 0.6995 - val_loss: 0.5846 - val_accuracy: 0.7805
Epoch 3/50
8/8 [==============================] - 0s 13ms/step - loss: 0.5902 - accuracy: 0.6995 - val_loss: 0.5842 - val_accuracy: 0.7805
Epoch 4/50
8/8 [==============================] - 0s 12ms/step - loss: 0.5953 - accuracy: 0.6995 - val_loss: 0.5839 - val_accuracy: 0.7805
Epoch 5/50
8/8 [==============================] - 0s 14ms/step - loss: 0.5962 - accuracy: 0.7049 - val_loss: 0.5836 - val_accuracy: 0.7805
Epoch 6/50
8/8 [==============================] - 0s 12ms/step - loss: 0.5970 - accuracy: 0.6967 - val_loss: 0.5833 - val_accuracy: 0.7805
Epoch 7/50
8/8 [==============================] - 0s 8ms/step - loss: 0.5939 - accuracy: 0.7022 - val_loss: 0.5830 - val_accuracy: 0.7805
Epoch 8/50
8/8 [============

In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score
y_pred = classifier.predict(X_newtest)
y_pred = y_pred > 0.5

print("Confusion matrix:")
print(confusion_matrix(y_test, y_pred))
print("Accuracy: {:.2f}%".format(accuracy_score(y_test, y_pred)*100))

6/6 [==============================] - 0s 3ms/step
Confusion matrix:
[[  0  48]
 [  0 127]]
Accuracy: 72.57%
